In [1]:
import findspark

findspark.init('C:/Users/Bruno/anaconda3/Lib/site-packages/Spark')

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, LongType
from pyspark.ml.recommendation import ALS
import sys
import codecs

def loadMovieNames():
    movieNames = {}
    # CHANGE THIS TO THE PATH TO YOUR u.ITEM FILE:
    with codecs.open("C:/Users/Bruno/Desktop/Python/Projectos Big Data/Spark/ml-100k/u.ITEM", "r", encoding='ISO-8859-1', errors='ignore') as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames


spark = SparkSession.builder.appName("ALSExample").getOrCreate()
    
moviesSchema = StructType([ \
                     StructField("userID", IntegerType(), True), \
                     StructField("movieID", IntegerType(), True), \
                     StructField("rating", IntegerType(), True), \
                     StructField("timestamp", LongType(), True)])
    
names = loadMovieNames()
    
ratings = spark.read.option("sep", "\t").schema(moviesSchema) \
    .csv("C:/Users/Bruno/Desktop/Python/Projectos Big Data/Spark/ml-100k/u.data")
    
print("Training recommendation model...")

als = ALS().setMaxIter(5).setRegParam(0.01).setUserCol("userID").setItemCol("movieID") \
    .setRatingCol("rating")
    
model = als.fit(ratings)

# Manually construct a dataframe of the user ID's we want recs for
#userID = int(sys.argv[1])
userID = 1

userSchema = StructType([StructField("userID", IntegerType(), True)])
users = spark.createDataFrame([[userID,]], userSchema)

recommendations = model.recommendForUserSubset(users, 10).collect()

print("Top 10 recommendations for user ID " + str(userID))

for userRecs in recommendations:
    myRecs = userRecs[1]  #userRecs is (userID, [Row(movieId, rating), Row(movieID, rating)...])
    for rec in myRecs: #my Recs is just the column of recs for the user
        movie = rec[0] #For each rec in the list, extract the movie ID and rating
        rating = rec[1]
        movieName = names[movie]
        print(movieName + str(rating))

Training recommendation model...
Top 10 recommendations for user ID 1
Secret Agent, The (1996)6.662696838378906
Living in Oblivion (1995)6.202996253967285
Crooklyn (1994)6.185708522796631
Faust (1994)6.109871864318848
Angel Baby (1995)6.007585525512695
Grosse Fatigue (1994)5.950167655944824
Two or Three Things I Know About Her (1966)5.802209854125977
Hard Eight (1996)5.7189717292785645
Unzipped (1995)5.713100433349609
So Dear to My Heart (1949)5.638084888458252
